In [11]:
import os
from datetime import datetime
import time

import numpy as np
import pandas as pd
from recbole.quick_start import run_recbole
from tqdm import tqdm
import torch
from recbole.quick_start import load_data_and_model


In [4]:
model_path='saved/NeuMF-Apr-14-2022_03-42-07.pth'

config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_path)

14 Apr 08:43    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /opt/ml/input/data/recbole
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.01
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni50'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user
ITEM_ID_FIELD = item
RATING_FIELD = rating
TI

In [ ]:
device = config.final_config_dict['device']

In [ ]:
# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr')
    
    #inferene용 데이터 준비
x_u = []
x_i = []

unseen = np.where(matrix.toarray()[:,:]==0)
unseen_u = unseen[0]
unseen_i = unseen[1]
    
x_u_torch = torch.IntTensor(unseen_u)
x_i_torch = torch.IntTensor(unseen_i)

x_u_torch = x_u_torch.to(device)
x_i_torch = x_i_torch.to(device)

    # batchsize만큼 split하기
batchsize = 800
u_splits = x_u_torch.split(batchsize, dim=0)
i_splits = x_i_torch.split(batchsize, dim=0)

    # user id, predict item id 저장 변수 
pred_list = []
user_list = []
item_list = []

model.eval()
for i in tqdm(range(len(u_splits))):
    batch_idx = i

    output2 = model(u_splits[batch_idx], i_splits[batch_idx])
    rating_pred = output2.cpu().data.numpy().copy() 

    batch_user_index = u_splits[batch_idx].cpu().numpy()
    batch_item_index = i_splits[batch_idx].cpu().numpy()

        # 예측값 저장
    pred_list.extend(rating_pred) 
    user_list.extend(batch_user_index)
    item_list.extend(batch_item_index)

100%|██████████| 260440/260440 [02:53<00:00, 1497.54it/s]


In [116]:
mat = np.zeros((31361, 6808))
indices = user_list, item_list#[:100]
values = pred_list#[:100]
mat[indices] = values

In [75]:
rating_result = pd.DataFrame(mat)
rating_result

,0,1,2,3,4,5,6,7,8,9,...,6798,6799,6800,6801,6802,6803,6804,6805,6806,6807
0,0.142713,0.366951,0.390093,0.357811,0.446566,0.405494,0.230517,0.649506,0.230211,0.262006,...,0.031585,0.026372,0.040435,0.023544,0.062308,0.037467,0.018648,0.023945,0.020548,0.021202
1,0.153798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010440,0.034056,0.109742,0.240704,0.025325,0.014526,0.025713,0.100689,0.244358,0.060024
2,0.111733,0.170024,0.078692,0.926692,0.980839,0.761739,0.062854,0.478494,0.217688,0.047748,...,0.138229,0.121553,0.011158,0.054484,0.276318,0.130255,0.001569,0.071782,0.002172,0.008480
3,0.111624,0.045660,0.050038,0.094176,0.147341,0.033902,0.031411,0.902989,0.112039,0.052812,...,0.060594,0.001808,0.002253,0.000566,0.080554,0.065041,0.016791,0.043681,0.003023,0.001438
4,0.042720,0.177855,0.779414,0.875504,0.516061,0.890451,0.425930,0.278061,0.784802,0.529356,...,0.000126,0.005091,0.016042,0.001303,0.136512,0.006500,0.000912,0.023336,0.000990,0.000630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31356,0.144449,0.483714,0.705730,0.678900,0.722507,0.811974,0.464096,0.552213,0.278222,0.624593,...,0.000833,0.022367,0.072918,0.042473,0.028095,0.001955,0.003268,0.004523,0.004012,0.004531
31357,0.082674,0.009169,0.014554,0.065116,0.175680,0.041352,0.031822,0.790197,0.005231,0.022732,...,0.311174,0.013258,0.024514,0.014205,0.166091,0.109404,0.001885,0.040114,0.009931,0.003810
31358,0.020041,0.325567,0.444455,0.267329,0.201282,0.629408,0.558214,0.419574,0.367360,0.617128,...,0.002868,0.005713,0.023300,0.003992,0.016066,0.001543,0.000836,0.060323,0.001350,0.000892
31359,0.007658,0.045646,0.189553,0.481046,0.085648,0.810277,0.025299,0.174826,0.087499,0.020689,...,0.001091,0.105474,0.002368,0.006673,0.203854,0.002243,0.000557,0.000577,0.000079,0.000383


In [80]:
pd.DataFrame(mat[1:,1:])


,0,1,2,3,4,5,6,7,8,9,...,6797,6798,6799,6800,6801,6802,6803,6804,6805,6806
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010440,0.034056,0.109742,0.240704,0.025325,0.014526,0.025713,0.100689,0.244358,0.060024
1,0.170024,0.078692,0.926692,0.980839,0.761739,0.062854,0.478494,0.217688,0.047748,0.021409,...,0.138229,0.121553,0.011158,0.054484,0.276318,0.130255,0.001569,0.071782,0.002172,0.008480
2,0.045660,0.050038,0.094176,0.147341,0.033902,0.031411,0.902989,0.112039,0.052812,0.017362,...,0.060594,0.001808,0.002253,0.000566,0.080554,0.065041,0.016791,0.043681,0.003023,0.001438
3,0.177855,0.779414,0.875504,0.516061,0.890451,0.425930,0.278061,0.784802,0.529356,0.191735,...,0.000126,0.005091,0.016042,0.001303,0.136512,0.006500,0.000912,0.023336,0.000990,0.000630
4,0.682295,0.229275,0.421460,0.605752,0.423021,0.506670,0.074781,0.085679,0.126277,0.135308,...,0.008536,0.337587,0.038157,0.295127,0.024542,0.004711,0.004135,0.004394,0.059277,0.232054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31355,0.483714,0.705730,0.678900,0.722507,0.811974,0.464096,0.552213,0.278222,0.624593,0.279109,...,0.000833,0.022367,0.072918,0.042473,0.028095,0.001955,0.003268,0.004523,0.004012,0.004531
31356,0.009169,0.014554,0.065116,0.175680,0.041352,0.031822,0.790197,0.005231,0.022732,0.001882,...,0.311174,0.013258,0.024514,0.014205,0.166091,0.109404,0.001885,0.040114,0.009931,0.003810
31357,0.325567,0.444455,0.267329,0.201282,0.629408,0.558214,0.419574,0.367360,0.617128,0.240729,...,0.002868,0.005713,0.023300,0.003992,0.016066,0.001543,0.000836,0.060323,0.001350,0.000892
31358,0.045646,0.189553,0.481046,0.085648,0.810277,0.025299,0.174826,0.087499,0.020689,0.022206,...,0.001091,0.105474,0.002368,0.006673,0.203854,0.002243,0.000557,0.000577,0.000079,0.000383


In [109]:
# 패딩부분 제외
#mat = mat[1:,1:]

In [117]:
import bottleneck as bn
idx = bn.argpartition(-mat, 10, axis=1)[:, :10]

In [118]:
user = []
item = []
for i_idx, arr_10 in enumerate(idx):
    user.extend([i_idx]*10)
    item.extend(arr_10)

u = pd.DataFrame(user, columns=['user'])
i = pd.DataFrame(item, columns=['item'])
all = pd.concat([u, i], axis=1)

In [119]:
user_id2token = dataset.field2id_token['user'] 
item_id2token = dataset.field2id_token['item']

In [121]:
def de_numerize(tp, re_p2id, re_s2id):
    uid = tp['user'].apply(lambda x: re_p2id[x])
    sid = tp['item'].apply(lambda x: re_s2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

ans = de_numerize(all, user_id2token, item_id2token)
ans.columns = ['user', 'item']

In [124]:
ans[10:]

,user,item
10,11,31696
11,11,8360
12,11,4370
13,11,4720
14,11,7438
...,...,...
313605,138493,47
313606,138493,32
313607,138493,318
313608,138493,4226
